In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
np.seterr(over='raise', invalid='raise', divide='raise');

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from helpers import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [25]:
from preprocessing import *
X0, X1, X2, X3, indices0, indices1, indices2, indices3 = separate_data(X, y)
y0 = y[indices0]
y1 = y[indices1]
y2 = y[indices2]
y3 = y[indices3]

In [41]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters = 5000
gamma = 0.000004
lambda_ = 0
batch_size = 100
k = 2
k_indices = build_k_indices(y0, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (y0[test_indices], X0[test_indices])

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (y0[training_indices], X0[training_indices])

    w_star = reg_logistic_regression(training_y, training_x, 0, gamma, max_iters)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))

Current iteration=0, the loss=33037.02215576352
Current iteration=1000, the loss=19566.28799291828
Current iteration=2000, the loss=19407.178228340526
Current iteration=3000, the loss=19383.457179752724
Current iteration=4000, the loss=19379.149998930752
The loss=19378.30739084321
Current iteration=0, the loss=33065.48966822366
Current iteration=1000, the loss=19903.427110723158
Current iteration=2000, the loss=19740.027664307323
Current iteration=3000, the loss=19714.065611711492
Current iteration=4000, the loss=19709.157408727595
The loss=19708.169444106512
Training error: 19543.23841747486
Test error: 19560.06296606074
Classification accuracy: 0.5141074145247818


### experiment 1

- max_iters = 10000
- gamma = 0.000004
- lambda_ = 0

Unless noted otherwise the parameters will not be changed between experiments

Training error: 96889.89798817722

Test error: 32303.189721963776

Classification accuracy: 0.5023500000000001

### experiment 2
Binarize -999 with 0 instead of ones.

Training error: 96889.89798817722

Test error: 32303.189721963776

Classification accuracy: 0.5023500000000001

No percetible change

### experiment 3
Remove feature 29 and 9 since they are sums of the other features

Training error: 96919.40308932838

Test error: 32313.056842448892

Classification accuracy: 0.502146

No percetible change meaning that these features are not useful

In [43]:
w0 = reg_logistic_regression(y0, X0, 0, gamma, max_iters)
w1 = reg_logistic_regression(y1, X1, 0, gamma, max_iters)
w2 = reg_logistic_regression(y2, X2, 0, gamma, max_iters)
w3 = reg_logistic_regression(y3, X3, 0, gamma, max_iters)

Current iteration=0, the loss=63292.66128433964
Current iteration=1000, the loss=39155.84902849459
Current iteration=2000, the loss=39097.34272966646
Current iteration=3000, the loss=39095.32788779734
Current iteration=4000, the loss=39095.247837776806
The loss=39095.244554673875
Current iteration=0, the loss=51613.252235173306
Current iteration=1000, the loss=41873.20322736096
Current iteration=2000, the loss=41871.1044812114
Current iteration=3000, the loss=41871.09276892613
Current iteration=4000, the loss=41871.09268833713
The loss=41871.09268775702
Current iteration=0, the loss=33099.01208931432
Current iteration=1000, the loss=25956.02678413775
Current iteration=2000, the loss=25931.95216460686
Current iteration=3000, the loss=25930.47256487579
Current iteration=4000, the loss=25930.36354819616
The loss=25930.35514608754
Current iteration=0, the loss=15186.765059592964
Current iteration=1000, the loss=11732.808674762226
Current iteration=2000, the loss=11700.696083988594
Current 

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_te = preprocess_inputs(tX_test)

In [ ]:
OUTPUT_PATH = '../submission.csv' # TODO: fill in desired name of output file for submission
y_pred = sigmoid(tX_te @ w)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = -1
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)